In [1]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_csv("cleanTweets5.csv")

,Unnamed: 0,text,timestamp,likes
0,1000,worth high low market cap billion,2013-07-04 19:00:36+00,1
1,1001,worth high low market cap billion,2013-07-04 20:00:36+00,1
2,1002,worth high low market cap billion,2013-07-04 21:00:35+00,1
3,1003,india cloud cover temp wind speed usher,2013-07-05 00:00:06+00,1
4,1004,worth high low market cap billion,2013-07-05 12:00:34+00,1
...,...,...,...,...
999993,1000993,girl asked finance new boob said listen sweeth...,2019-08-18 09:57:56+00,6
999994,1000994,bond bubble burst warns fall sure umbrella rai...,2019-08-18 09:58:10+00,8
999995,1000995,looking likely killer open database verified u...,2019-08-18 09:58:17+00,20
999996,1000996,NaN,2019-08-18 09:58:26+00,44


In [21]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/joshuaathayde/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [23]:
sid = SentimentIntensityAnalyzer()

In [24]:
sid.polarity_scores("this is good")

{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}

In [28]:
sid.polarity_scores(df['text'][0])

{'neg': 0.263, 'neu': 0.5, 'pos': 0.237, 'compound': -0.0516}

In [31]:
df['text'] = df['text'].map(str)

In [32]:
df['vader_sentiment'] = df['text'].map(sid.polarity_scores)

In [35]:
df['vader_sentiment'][1000]

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [41]:
df['weight_avg'] = list(map(lambda x, y: (x['pos'] - x['neg']) * y, df['vader_sentiment'], df['likes']))

In [42]:
df['weight_avg'][0]

-0.026000000000000023

In [43]:
df.to_csv('weighted_avg.csv')

In [45]:
df['timestamp'][0]

'2013-07-04 19:00:36+00'

In [55]:
df['date'] = list(map(lambda x: x.split(' ')[0], df['timestamp']))

In [56]:
import datetime

In [57]:
test = df['date'][0].split('-')

In [58]:
test

['2013', '07', '04']

In [60]:
test_date = datetime.date(int(test[0]), int(test[1]), int(test[2]))

In [61]:
test_date

datetime.date(2013, 7, 4)

In [94]:
def toDate(date):
    x = date.split('-')
    try: return datetime.date(int(x[0]), int(x[1]), int(x[2]))
    except: print(x)

In [95]:
df['date_obj'] = df['date'].map(toDate)

In [96]:
df['date_obj']

0         2013-07-04
1         2013-07-04
2         2013-07-04
3         2013-07-05
4         2013-07-05
             ...    
999993    2019-08-18
999994    2019-08-18
999995    2019-08-18
999996    2019-08-18
999997    2019-08-18
Name: date_obj, Length: 999998, dtype: object

In [64]:
col = df['date']

In [75]:
test = col.map(lambda x: x.split('-')[1])

In [87]:
int(test.max())

12

In [88]:
retval = False
for x in df['date'].isna():
    retval = retval or x
retval

False

In [82]:
df['date']

0         2013-07-04
1         2013-07-04
2         2013-07-04
3         2013-07-05
4         2013-07-05
             ...    
999993    2019-08-18
999994    2019-08-18
999995    2019-08-18
999996    2019-08-18
999997    2019-08-18
Name: date, Length: 999998, dtype: object

In [145]:
newdf = df.groupby('date_obj').sum()[['weight_avg']]

In [149]:
df2 = pd.DataFrame(newdf['weight_avg'])

In [169]:
df2.to_csv('signal_data.csv')

In [170]:
df = pd.read_csv("signal_data.csv", index = 'date_obj')

TypeError: read_csv() got an unexpected keyword argument 'index'

In [162]:
df['weight_avg']

0          -0.078
1          -0.006
2          -0.234
3           0.334
4          -0.182
          ...    
2228    27971.888
2229    24192.185
2230    17782.938
2231    12829.994
2232     2192.757
Name: weight_avg, Length: 2233, dtype: float64

In [174]:
df.rename(columns = {'date_obj': 'date'}, inplace = True)

In [175]:
df

,date,weight_avg
0,2013-07-04,-0.078
1,2013-07-05,-0.006
2,2013-07-06,-0.234
3,2013-07-07,0.334
4,2013-07-08,-0.182
...,...,...
2228,2019-08-14,27971.888
2229,2019-08-15,24192.185
2230,2019-08-16,17782.938
2231,2019-08-17,12829.994


In [176]:
df.to_csv('signal_data.csv')

In [177]:
df['date']

0       2013-07-04
1       2013-07-05
2       2013-07-06
3       2013-07-07
4       2013-07-08
           ...    
2228    2019-08-14
2229    2019-08-15
2230    2019-08-16
2231    2019-08-17
2232    2019-08-18
Name: date, Length: 2233, dtype: object